In [49]:
import os
import pandas as pd
import fastparquet
import snappy
import wordcloud
import spacy
import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
os.environ["PYTHONIOENCODING"] = "utf-8"

In [51]:
PATH = 'F:\\Sistema\\Downloads\\bigdatamusic_datas\\pontos_turisticos\\'

In [52]:
all_files = [f'{PATH}{file_name}' for file_name in os.listdir(PATH) if '.csv' in file_name]
all_files

['F:\\Sistema\\Downloads\\bigdatamusic_datas\\pontos_turisticos\\feiraslivres.csv',
 'F:\\Sistema\\Downloads\\bigdatamusic_datas\\pontos_turisticos\\mercadospublicos.csv',
 'F:\\Sistema\\Downloads\\bigdatamusic_datas\\pontos_turisticos\\museu-1.csv',
 'F:\\Sistema\\Downloads\\bigdatamusic_datas\\pontos_turisticos\\pontesdorecife.csv',
 'F:\\Sistema\\Downloads\\bigdatamusic_datas\\pontos_turisticos\\shopping.csv',
 'F:\\Sistema\\Downloads\\bigdatamusic_datas\\pontos_turisticos\\teatros.csv']

In [53]:
mercadospublicos = pd.read_csv(all_files[1], sep=';')
mercadospublicos = mercadospublicos.drop(["latitude", "descricao", "longitude"], axis=1)
 
pontesdorecife = pd.read_csv(all_files[3], sep=';')
pontesdorecife = pontesdorecife.drop(["latitude", "descricao", "longitude"], axis=1)
pontesdorecife =pontesdorecife.rename(columns={"Nome": "nome"})
 
teatros = pd.read_csv(all_files[5], sep=';')
teatros = teatros.drop(["logradouro","Telefone", "descricao","latitude","longitude"], axis=1)
 
museu = pd.read_csv(all_files[2], sep=';')
museu = museu.drop(["logradouro","latitude", "descricao","longitude","telefone","site"], axis=1)
 
#feiraslivres = pd.read_csv(all_files[0], sep=';')
#feiraslivres = feiraslivres.drop(["latitude", "longitude"], axis=1)
 
shopping = pd.read_csv(all_files[4], sep=';')
shopping = shopping.drop(["logradouro","latitude", "descricao","longitude","telefone","site","funcionamento","funcionamentoDomingo"], axis=1)
 
all_datas = pd.concat([mercadospublicos, pontesdorecife, teatros, museu, shopping], sort=False).reset_index(drop=False)
all_datas = all_datas.sort_values(['bairro','nome']).drop(["index"], axis=1)
 

In [55]:
count_places = all_datas[['bairro','nome']].groupby(['bairro'], as_index=False).count()
count_places

,bairro,nome
0,Santo Antônio,1
1,Apipucos,1
2,Boa Viagem,2
3,Boa Vista,7
4,Casa Amarela,1
5,Casa Forte,2
6,Cordeiro,1
7,Encruzilhada,1
8,Graças,1
9,Jaqueira,1


In [56]:
#all_datas = all_datas[['bairro','nome']].groupby(['bairro'], as_index=False).count()

all_datas = all_datas[['bairro','nome']].groupby(['bairro'], as_index=False).agg(lambda x: ' -- '.join(x))

"""
all_datas['nome'] = all_datas[['bairro','nome']].groupby(['bairro'],as_index=False)['nome'].transform(lambda x: ' -- '.join(x))
all_datas = all_datas.drop_duplicates()
"""
all_datas

,bairro,nome
0,Santo Antônio,Museu Franciscano de Arte Sacra
1,Apipucos,Casa-Museu Magdalena e Gilberto Freyre
2,Boa Viagem,Shopping Recife -- Teatro Luiz Mendonça
3,Boa Vista,Instituto Arqueológico Histórico e Geográfico ...
4,Casa Amarela,Mercado de Casa Amarela
5,Casa Forte,Museu do Homem do Nordeste -- Plaza Casa Forte
6,Cordeiro,Mercado do Cordeiro
7,Encruzilhada,Mercado da Encruzilhada
8,Graças,Museu do Estado de Pernambuco
9,Jaqueira,Executive Trade Cemter


In [57]:
all_datas.to_parquet(f'{PATH}final_data\\turistic_points.parquet', engine='fastparquet')